In [5]:
from ads_api import fb_ad_api
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import ast

In [4]:
ad_results = fb_ad_api()
#call the API and get 10 pages of 100 results each
ad_results.call_api(page_limit=100)
ad_df = ad_results.make_df()

Pulling page:1
Pulling page:2
Pulling page:3
Pulling page:4
Pulling page:5
Pulling page:6
Pulling page:7
Pulling page:8
Pulling page:9
Pulling page:10


,AdID,AdURL,AdText,HostedPage,Impressions,Currency,AdSpending
0,161547868597338,https://www.facebook.com/ads/archive/render_ad...,{{product.brand}},Vote Yes for Fair Tax,"{'lower_bound': '4000', 'upper_bound': '4999'}",USD,"{'lower_bound': '100', 'upper_bound': '199'}"
1,231634031267946,https://www.facebook.com/ads/archive/render_ad...,{{product.brand}},Vote Yes for Fair Tax,"{'lower_bound': '5000', 'upper_bound': '5999'}",USD,"{'lower_bound': '100', 'upper_bound': '199'}"
2,233098731244956,https://www.facebook.com/ads/archive/render_ad...,{{product.brand}},Vote Yes for Fair Tax,"{'lower_bound': '8000', 'upper_bound': '8999'}",USD,"{'lower_bound': '100', 'upper_bound': '199'}"
3,246740140011738,https://www.facebook.com/ads/archive/render_ad...,{{product.brand}},Vote Yes for Fair Tax,"{'lower_bound': '6000', 'upper_bound': '6999'}",USD,"{'lower_bound': '100', 'upper_bound': '199'}"
4,290585385271290,https://www.facebook.com/ads/archive/render_ad...,{{product.brand}},Vote Yes for Fair Tax,"{'lower_bound': '4000', 'upper_bound': '4999'}",USD,"{'lower_bound': '0', 'upper_bound': '99'}"


In [29]:
# change upper/lower bound data entries to just upper_bound value
def remove_dict(sample):
    sample_dict = ast.literal_eval(str(sample))
    return sample_dict['upper_bound']
ad_df['Impressions'] = ad_df['Impressions'].map(remove_dict)
ad_df['AdSpending'] = ad_df['AdSpending'].map(remove_dict)

TypeError: 'int' object is not subscriptable

In [31]:
ad_df.head()

,AdID,AdURL,AdText,HostedPage,Impressions,Currency,AdSpending
0,161547868597338,https://www.facebook.com/ads/archive/render_ad...,{{product.brand}},Vote Yes for Fair Tax,4999,USD,199
1,231634031267946,https://www.facebook.com/ads/archive/render_ad...,{{product.brand}},Vote Yes for Fair Tax,5999,USD,199
2,233098731244956,https://www.facebook.com/ads/archive/render_ad...,{{product.brand}},Vote Yes for Fair Tax,8999,USD,199
3,246740140011738,https://www.facebook.com/ads/archive/render_ad...,{{product.brand}},Vote Yes for Fair Tax,6999,USD,199
4,290585385271290,https://www.facebook.com/ads/archive/render_ad...,{{product.brand}},Vote Yes for Fair Tax,4999,USD,99


In [6]:
#load the imdb dataset for the positivity tokenizer
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)
imdb_tokenizer = info.features['text'].encoder


In [8]:
#load model for toxicity
ad_features = ['positivity','identityAttack', 'insult', 'toxicity']
model_files = [f'../ml_settings/{feature}_rnn.h5' for feature in ad_features]
model_files

['../ml_settings/positivity_rnn.h5',
 '../ml_settings/identity_attack_rnn.h5',
 '../ml_settings/insult_rnn.h5',
 '../ml_settings/toxicity_rnn.h5']

In [38]:
model = tf.keras.models.load_model('../ml_settings/toxicity_rnn.h5')

In [ ]:
#function to return a ML prediction from text
def sample_predict(sample_pred_text, pad=True):
  encoded_sample_pred_text = tokenizer.encode(sample_pred_text) #encode sample

  if pad: #pads length with 0s to 64 entries if shorter than 64 entries
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))

  return (predictions)

# mapping function for the 
def binary_label(prediction):
    if prediction >= 0.5:
        return 1
    return 0

In [36]:
#loop for each of the ad feature models
for feature in ad_features:
    #set specific tokenizer if doing positivity predictions
    print(feature)
    if feature == 'positivity':
        tokenizer = imdb_tokenizer
    else:
        tokenizer = tfds.features.text.SubwordTextEncoder.load_from_file('../ml_settings/large_tokenizer.subwords')
    #load model
    model = tf.keras.models.load_model(f'../ml_settings/{feature}_rnn.h5')

    #map predictions to new column
    ad_df[feature] = ad_df['AdText'].map(lambda x: binary_label(sample_predict(x)))
    

positivity


KeyError: 'sample_weight_mode'